<a href="https://colab.research.google.com/github/Drsrina/MongoAPI/blob/main/C%C3%B3pia_de_MONGO_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio do Maluco

In [ ]:
# @title ☢️ LIGAR REATOR (Token Pré-Configurado)
# @markdown Reinstala tudo, limpa conflitos e sobe o servidor com seu Token.

import os
import subprocess
import sys
import time

# --- 1. CONFIGURAÇÃO DE SOBREVIVÊNCIA ---
# Para manter seus dados seguros, configure suas chaves no Google Colab Secrets
# (o ícone de chave 🔑 na barra lateral esquerda).
# As chaves devem ter os nomes: NGROK_TOKEN, MONGO_USER, MONGO_PASS, MONGO_HOST.
# Se não estiverem em Secrets, ele tentará carregar de variáveis de ambiente.

NGROK_TOKEN = os.getenv("NGROK_TOKEN", "") # Deixe vazio se não for usar ngrok
MONGO_USER = os.getenv("MONGO_USER", "")
MONGO_PASS = os.getenv("MONGO_PASS", "")
MONGO_HOST = os.getenv("MONGO_HOST", "")

if not all([MONGO_USER, MONGO_PASS, MONGO_HOST]):
    print("⚠️ ATENÇÃO: Credenciais do MongoDB não configuradas. Verifique suas variáveis de ambiente ou Colab Secrets.")

print("🧹 Limpando a área de cirurgia (Desinstalando conflitos)...")
# Removemos o pacote 'bson' pirata que quebra o pymongo
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "bson"])

print("💊 Injetando o soro correto (Instalando dependências)...")
# Instalamos apenas o necessário. O 'pymongo' já traz o bson original.
libs = ["strawberry-graphql", "pymongo", "fastapi", "uvicorn", "pymongo", "certifi", "pyngrok"]
for lib in libs:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
    except:
        pass # Ignora se já estiver instalado

print("✅ Dependências estabilizadas.")

# Importamos AGORA que temos certeza que existem
from pyngrok import ngrok, conf

# --- 2. LIMPEZA DE PROCESSOS ---
subprocess.run(["pkill", "-f", "uvicorn"])
subprocess.run(["pkill", "-f", "ngrok"])

# --- 3. SUBIR O SERVIDOR ---
if NGROK_TOKEN:
    conf.get_default().auth_token = NGROK_TOKEN

# Prepara o ambiente com as senhas
my_env = os.environ.copy()
my_env["MONGO_USER"] = MONGO_USER
my_env["MONGO_PASS"] = MONGO_PASS
my_env["MONGO_HOST"] = MONGO_HOST

print(f"🧠 Reiniciando o Cérebro em IPv4 (127.0.0.1)...")

# Roda o Uvicorn forçando localhost (evita erro de conexão recusada)
proc = subprocess.Popen(
    ["uvicorn", "api:app", "--host", "127.0.0.1", "--port", "8000"],
    env=my_env
)

time.sleep(5)

# Verifica se o paciente sobreviveu aos primeiros 5 segundos
if proc.poll() is not None:
    print("💀 O SERVIDOR CAIU IMEDIATAMENTE! Verifique o api.py")
else:
    try:
        ngrok.kill()
        # Abre o túnel explicitamente para HTTP no localhost
        url = ngrok.connect("127.0.0.1:8000", "http").public_url

        print("\n" + "🟢"*35)
        print(f"   SISTEMA ONLINE! ACESSE: {url}/graphql")
        print("🟢"*35 + "\n")

        # Mantém vivo
        while True: time.sleep(1)

    except KeyboardInterrupt:
        ngrok.kill()
        proc.terminate()

🧹 Limpando a área de cirurgia (Desinstalando conflitos)...
💊 Injetando o soro correto (Instalando dependências)...
✅ Dependências estabilizadas.
🧠 Reiniciando o Cérebro em IPv4 (127.0.0.1)...

🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢
   SISTEMA ONLINE! ACESSE: https://charlena-irresponsive-suppositionally.ngrok-free.dev/graphql
🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢



# PRÉ LAB

In [ ]:
# @title 🧨 Derrubar Servidor Antigo
# @markdown Isso mata qualquer processo do Python/Uvicorn rodando solto para liberarmos a porta.
!pkill -f uvicorn
!pkill -f localtunnel
print("💀 Processos antigos eliminados. A porta 8000 está livre.")

💀 Processos antigos eliminados. A porta 8000 está livre.


# NEW LAB 3

In [ ]:
# @title 🕵️ iniciar o Detetive

import pymongo
import certifi
import os
import urllib.parse
import json
from bson import json_util

# 1. CONFIGURAÇÃO E CONEXÃO
print("🕵️ Iniciando o Detetive de Schema...")

raw_user = os.getenv("MONGO_USER")
raw_pass = os.getenv("MONGO_PASS")
raw_host = os.getenv("MONGO_HOST")

if not all([raw_user, raw_pass, raw_host]):
    raise ValueError("Credenciais do MongoDB (MONGO_USER, MONGO_PASS, MONGO_HOST) não configuradas. Verifique suas variáveis de ambiente ou Colab Secrets.")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{user}:{passwd}@{raw_host}/?retryWrites=true&w=majority&authSource=admin"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]
    # Pegamos 50 documentos para ter uma amostra boa
    sample_docs = list(coll.find().sort("SP_Date_Time", -1).limit(5000))
    print(f"✅ Conectado! Analisando {len(sample_docs)} documentos...")
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    sample_docs = []

# 2. FUNÇÕES DE ANÁLISE (O CÉREBRO)

def to_snake_case(text):
    """Converte CamelCase ou chaves estranhas para snake_case do Python"""
    import re
    str1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', str1).lower().replace(" ", "_").replace("-", "_")

def analyze_structure(docs):
    """Cria um super-schema combinando todos os campos encontrados"""
    schema = {}

    for doc in docs:
        for key, value in doc.items():
            if key == "_id": continue # Tratamos ID separado

            field_type = "Optional[str]" # Default seguro
            is_nested = False
            nested_schema = {}

            if isinstance(value, bool):
                field_type = "Optional[bool]"
            elif isinstance(value, int):
                field_type = "Optional[int]"
            elif isinstance(value, float):
                field_type = "Optional[float]"
            elif isinstance(value, dict):
                field_type = "Nested"
                is_nested = True
                nested_schema = value
            elif isinstance(value, list):
                field_type = "Optional[List[str]]" # Simplificação para listas

            # Se já existe e é nested, precisamos mergir o schema interno
            if key in schema and schema[key]['is_nested'] and is_nested:
                # Merge recursivo simples (nao implementado full aqui para brevidade, pega o ultimo)
                schema[key]['nested_sample'].update(nested_schema)
            elif key not in schema:
                schema[key] = {
                    "original_key": key,
                    "snake_key": to_snake_case(key),
                    "type": field_type,
                    "is_nested": is_nested,
                    "nested_sample": nested_schema if is_nested else {}
                }
    return schema

# 3. GERADOR DE CÓDIGO PYTHON

def generate_nested_classes(schema, parent_name=""):
    code_blocks = ""
    for key, info in schema.items():
        if info['is_nested']:
            class_name = f"{key.capitalize()}Data"
            # Recursão para analisar sub-níveis se tivermos tempo,
            # aqui vamos usar a amostra direta para criar a classe
            sub_schema = analyze_structure([info['nested_sample']])

            # Gera a classe filha primeiro
            code_blocks += generate_nested_classes(sub_schema, class_name)

            # Define a classe atual
            code_blocks += f"\n@strawberry.type\nclass {class_name}:\n"
            for sub_k, sub_info in sub_schema.items():
                code_blocks += f"    {sub_info['snake_key']}: {sub_info['type']}\n"

            # Atualiza o tipo no pai para usar o nome da classe
            info['type'] = f"Optional[{class_name}]"
            info['class_name'] = class_name # Guardar para uso no resolver
            info['sub_schema'] = sub_schema

    return code_blocks

def generate_resolver_logic(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            # Lógica complexa para instanciar classes aninhadas
            sub_cls = info['class_name']
            sub_dict_name = f"{info['snake_key']}_dict"
            lines.append(f"        {sub_dict_name} = {dict_var_name}.get('{info['original_key']}') or {{}}")
            lines.append(f"        {info['snake_key']}_obj = {sub_cls}(")

            for sub_k, sub_info in info['sub_schema'].items():
                 lines.append(f"            {sub_info['snake_key']}={sub_dict_name}.get('{sub_info['original_key']}'),")

            lines.append("        )")
        else:
            pass # Primitivos são diretos no return

    return "\n".join(lines)

def generate_return_fields(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            lines.append(f"            {info['snake_key']}={info['snake_key']}_obj,")
        else:
            lines.append(f"            {info['snake_key']}={dict_var_name}.get('{info['original_key']}'),")
    return "\n".join(lines)

# --- EXECUÇÃO DO GERADOR ---

if not sample_docs:
    print("❌ Sem dados para analisar. Abortando.")
else:
    full_schema = analyze_structure(sample_docs)
    nested_classes_code = generate_nested_classes(full_schema)

    # Montagem do arquivo final
    final_code = f"""
import strawberry
from fastapi import FastAPI
from strawberry.fastapi import GraphQLRouter
from fastapi.middleware.cors import CORSMiddleware
import pymongo
import certifi
from typing import List, Optional
import json
import os
import urllib.parse
from bson import json_util

# --- CONEXÃO AUTOMÁTICA ---
raw_user = os.getenv("MONGO_USER", "luludapomerania")
raw_pass = os.getenv("MONGO_PASS", "luludapomerania")
raw_host = os.getenv("MONGO_HOST", "clustercaipora.ewcfpo1.mongodb.net")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{{user}}:{{passwd}}@{{raw_host}}/?retryWrites=true&w=majority&authSource=admin"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]
except:
    pass

@strawberry.scalar
class JSON:
    @staticmethod
    def serialize(value) -> any:
        return value
    @staticmethod
    def parse_value(value) -> any:
        return value

# --- CLASSES GERADAS PELO DETETIVE ---
{nested_classes_code}
@strawberry.type
class DocumentoAutoMapeado:
    mongo_id: str
"""
    # Adiciona campos na classe principal
    for key, info in full_schema.items():
        final_code += f"    {info['snake_key']}: {info['type']}\n"

    final_code += """
    conteudo_completo: JSON

def get_dados(limit: int = 10) -> List[DocumentoAutoMapeado]:
    cursor = coll.find().sort("SP_Date_Time", -1).limit(limit)
    res = []
    for doc in cursor:
        doc_str = json_util.dumps(doc)
        doc_dict = json.loads(doc_str)

        # Lógica de preenchimento automático
"""
    final_code += generate_resolver_logic(full_schema)

    final_code += """
        res.append(DocumentoAutoMapeado(
            mongo_id=str(doc.get('_id', '')),
"""
    final_code += generate_return_fields(full_schema)

    final_code += """
            conteudo_completo=doc_dict
        ))
    return res

@strawberry.type
class Query:
    @strawberry.field
    def explorar_dados(self, limit: int = 10) -> List[DocumentoAutoMapeado]:
        return get_dados(limit)

schema = strawberry.Schema(query=Query)
graphql_app = GraphQLRouter(schema)
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(graphql_app, prefix="/graphql")
"""

    # Salvando o arquivo
    with open("api.py", "w") as f:
        f.write(final_code)

    print("✨ SUCESSO! Arquivo 'api.py' foi gerado com base nos seus dados.")
    print("👉 Agora rode a próxima célula para iniciar o servidor.")

🕵️ Iniciando o Detetive de Schema...
✅ Conectado! Analisando 5000 documentos...
✨ SUCESSO! Arquivo 'api.py' foi gerado com base nos seus dados.
👉 Agora rode a próxima célula para iniciar o servidor.


In [ ]:
# @title 🧹 Detetive Final (Com Faxina de Datas)
import pymongo
import certifi
import os
import urllib.parse
import json
import re
import keyword
from bson import json_util

# --- 1. CONFIGURAÇÃO ---
print("🧹 Iniciando Detetive com Faxina de Datas...")

raw_user = os.getenv("MONGO_USER")
raw_pass = os.getenv("MONGO_PASS")
raw_host = os.getenv("MONGO_HOST")

if not all([raw_user, raw_pass, raw_host]):
    raise ValueError("Credenciais do MongoDB (MONGO_USER, MONGO_PASS, MONGO_HOST) não configuradas. Verifique suas variáveis de ambiente ou Colab Secrets.")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{user}:{passwd}@{raw_host}/?retryWrites=true&w=majority"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]

    # Mantendo sua lógica de Sort
    print("⏳ Buscando amostra...")
    sample_docs = list(coll.find().sort("SP_Date_Time", -1).limit(5000))
    print(f"✅ Conectado! Analisando {len(sample_docs)} documentos...")
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    sample_docs = []

# --- 2. FUNÇÕES DE ANÁLISE ---

def to_snake_case(text):
    str1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
    clean = re.sub('([a-z0-9])([A-Z])', r'\1_\2', str1).lower().replace(" ", "_").replace("-", "_")
    clean = re.sub(r'[^a-z0-9_]', '', clean)
    if clean and clean[0].isdigit(): return f"field_{clean}"
    if keyword.iskeyword(clean): return f"{clean}_field"
    return clean

def analyze_structure(docs):
    schema = {}
    for doc in docs:
        for key, value in doc.items():
            if key == "_id": continue
            snake_key = to_snake_case(key)
            if not snake_key: continue

            # Initialize these variables with default values
            field_type = "Optional[str]" # Default safe type
            is_nested = False
            nested_schema = {}

            # Se for dict com chaves de BSON ($date, $oid), trata como string direta
            if isinstance(value, dict) and ('$date' in value or '$oid' in value):
                 field_type = "Optional[str]"
                 is_nested = False
                 nested_schema = {}
            elif isinstance(value, dict):
                field_type = "Nested"
                is_nested = True
                nested_schema = value
            elif isinstance(value, bool): field_type = "Optional[bool]"
            elif isinstance(value, int): field_type = "Optional[int]"
            elif isinstance(value, float): field_type = "Optional[float]"
            elif isinstance(value, list): field_type = "Optional[List[str]]"
            # No `else` needed here as `field_type` is initialized to "Optional[str]"

            if key not in schema:
                schema[key] = {
                    "original_key": key,
                    "snake_key": snake_key,
                    "type": field_type,
                    "is_nested": is_nested,
                    "nested_sample": nested_schema if is_nested else {}
                }
            elif is_nested and schema[key]['is_nested']:
                schema[key]['nested_sample'].update(nested_schema)
    return schema

# --- 3. GERADOR ---

def generate_nested_classes(schema):
    code_blocks = ""
    for key, info in schema.items():
        if info['is_nested']:
            class_name = f"{info['snake_key'].capitalize()}Data"
            sub_schema = analyze_structure([info['nested_sample']])
            if not sub_schema:
                info['is_nested'] = False
                info['type'] = "Optional[JSON]"
                continue
            code_blocks += generate_nested_classes(sub_schema)
            code_blocks += f"\n@strawberry.type\nclass {class_name}:\n"
            for sub_k, sub_info in sub_schema.items():
                code_blocks += f"    {sub_info['snake_key']}: {sub_info['type']}\n"
            info['type'] = f"Optional[{class_name}]"
            info['class_name'] = class_name
            info['sub_schema'] = sub_schema
    return code_blocks

def generate_resolver_logic(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            sub_cls = info['class_name']
            sub_dict_name = f"{info['snake_key']}_dict"
            lines.append(f"        val_{key} = {dict_var_name}.get('{info['original_key']}')")
            lines.append(f"        {sub_dict_name} = val_{key} if isinstance(val_{key}, dict) else {{}}")
            lines.append(f"        {info['snake_key']}_obj = {sub_cls}(")
            for sub_k, sub_info in info['sub_schema'].items():
                 lines.append(f"            {sub_info['snake_key']}={sub_dict_name}.get('{sub_info['original_key']}'),")
            lines.append("        )")
    return "\n".join(lines)

def generate_return_fields(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            lines.append(f"            {info['snake_key']}={info['snake_key']}_obj,")
        else:
            lines.append(f"            {info['snake_key']}={dict_var_name}.get('{info['original_key']}'),")
    return "\n".join(lines)

# --- EXECUÇÃO ---

if not sample_docs:
    print("❌ Sem dados.")
else:
    full_schema = analyze_structure(sample_docs)
    nested_classes_code = generate_nested_classes(full_schema)

    final_code = f"""
import strawberry
from fastapi import FastAPI
from strawberry.fastapi import GraphQLRouter
from fastapi.middleware.cors import CORSMiddleware
import pymongo
import certifi
from typing import List, Optional, Any
import json
import os
import urllib.parse
from bson import json_util

raw_user = os.getenv("MONGO_USER")
raw_pass = os.getenv("MONGO_PASS")
raw_host = os.getenv("MONGO_HOST")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{{user}}:{{passwd}}@{{raw_host}}/?retryWrites=true&w=majority"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]
except:
    pass

@strawberry.scalar
class JSON:
    @staticmethod
    def serialize(value) -> Any: return value
    @staticmethod
    def parse_value(value) -> Any: return value

# --- FUNÇÃO FAXINEIRA (CORRIGE AS DATAS) ---
def sanitize_mongo(data):
    if isinstance(data, dict):
        if "$date" in data: return data["$date"] # Pega só a string da data
        if "$oid" in data: return data["$oid"]   # Pega só o ID
        return {{k: sanitize_mongo(v) for k, v in data.items()}}
    elif isinstance(data, list):
        return [sanitize_mongo(i) for i in data]
    return data

{nested_classes_code}

@strawberry.type
class DocumentoOficial:
    mongo_id: str
"""
    for key, info in full_schema.items():
        final_code += f"    {info['snake_key']}: {info['type']}\n"

    final_code += """
    conteudo_completo: JSON

def get_dados(limit: int = 10):
    cursor = coll.find().sort("SP_Date_Time", -1).limit(limit)
    res = []
    for doc in cursor:
        # Serializa para BSON string
        doc_str = json_util.dumps(doc)
        # Carrega e FAZ A FAXINA nas datas
        doc_dict = sanitize_mongo(json.loads(doc_str))

"""
    final_code += generate_resolver_logic(full_schema)
    final_code += """
        res.append(DocumentoOficial(
            mongo_id=str(doc.get('_id', '')),
"""
    final_code += generate_return_fields(full_schema)
    final_code += """
            conteudo_completo=doc_dict
        ))
    return res

@strawberry.type
class Query:
    @strawberry.field
    def explorar_dados(self, limit: int = 10) -> List[DocumentoOficial]:
        return get_dados(limit)

schema = strawberry.Schema(query=Query)
graphql_app = GraphQLRouter(schema)
app = FastAPI()
app.include_router(graphql_app, prefix="/graphql")
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"], allow_credentials=True)
"""

    with open("api.py", "w") as f:
        f.write(final_code)

    print("\n✅ API.PY com FAXINA DE DATAS gerado!")
    print("👉 Agora rode o 'LIGAR REATOR' e os erros de data vão sumir.")

🧹 Iniciando Detetive com Faxina de Datas...
⏳ Buscando amostra...
✅ Conectado! Analisando 5000 documentos...

✅ API.PY com FAXINA DE DATAS gerado!
👉 Agora rode o 'LIGAR REATOR' e os erros de data vão sumir.


In [ ]:
# @title 🔧 API Blindada (Fix: Scalar JSON Oficial + Cursor)
import pymongo
import certifi
import os
import urllib.parse
import json
import re
import keyword
from bson import json_util

# --- 1. CONFIGURAÇÃO ---
print("🔧 Gerando API com JSON Nativo e Cursor...")

raw_user = os.getenv("MONGO_USER")
raw_pass = os.getenv("MONGO_PASS")
raw_host = os.getenv("MONGO_HOST")

if not all([raw_user, raw_pass, raw_host]):
    raise ValueError("Credenciais do MongoDB (MONGO_USER, MONGO_PASS, MONGO_HOST) não configuradas. Verifique suas variáveis de ambiente ou Colab Secrets.")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{user}:{passwd}@{raw_host}/?retryWrites=true&w=majority"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]
    # Pega 100 docs para o schema
    sample_docs = list(coll.find().limit(10000))
except:
    sample_docs = []

# --- 2. FUNÇÕES DE ANÁLISE ---

def to_snake_case(text):
    str1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', text)
    clean = re.sub('([a-z0-9])([A-Z])', r'\1_\2', str1).lower().replace(" ", "_").replace("-", "_")
    clean = re.sub(r'[^a-z0-9_]', '', clean)
    if not clean: return "campo_vazio"
    if clean[0].isdigit(): return f"field_{clean}"
    if keyword.iskeyword(clean): return f"{clean}_field"
    return clean

def analyze_structure(docs):
    schema = {}
    for doc in docs:
        for key, value in doc.items():
            if key == "_id": continue
            snake_key = to_snake_case(key)
            if not snake_key: continue

            # Lógica de detecção de tipo
            field_type = "Optional[str]"
            is_nested = False
            nested_schema = {}

            # Se for dict especial do Mongo ($date, $oid), vira string
            if isinstance(value, dict) and ('$date' in value or '$oid' in value):
                 field_type = "Optional[str]"
            elif isinstance(value, dict):
                field_type = "Nested"
                is_nested = True
                nested_schema = value
            elif isinstance(value, bool): field_type = "Optional[bool]"
            elif isinstance(value, int): field_type = "Optional[int]"
            elif isinstance(value, float): field_type = "Optional[float]"
            elif isinstance(value, list): field_type = "Optional[List[str]]" # Listas viram strings simples por segurança

            if key not in schema:
                schema[key] = {
                    "original_key": key,
                    "snake_key": snake_key,
                    "type": field_type,
                    "is_nested": is_nested,
                    "nested_sample": nested_schema if is_nested else {}
                }
            elif is_nested and schema[key]['is_nested']:
                # Merge de estruturas aninhadas
                schema[key]['nested_sample'].update(nested_schema)

    return schema

# --- 3. GERADOR DO CÓDIGO FINAL ---

def generate_nested_classes(schema):
    code_blocks = ""
    for key, info in schema.items():
        if info['is_nested']:
            class_name = f"{info['snake_key'].capitalize()}Data"
            sub_schema = analyze_structure([info['nested_sample']])

            # Se o sub-schema for vazio ou muito complexo, vira JSON
            if not sub_schema:
                info['is_nested'] = False
                info['type'] = "Optional[JSON]" # Usa JSON nativo
                continue

            code_blocks += generate_nested_classes(sub_schema)
            code_blocks += f"\n@strawberry.type\nclass {class_name}:\n"
            for sub_k, sub_info in sub_schema.items():
                code_blocks += f"    {sub_info['snake_key']}: {sub_info['type']}\n"

            info['type'] = f"Optional[{class_name}]"
            info['class_name'] = class_name
            info['sub_schema'] = sub_schema
    return code_blocks

def generate_resolver_logic(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            sub_cls = info['class_name']
            sub_dict_name = f"{info['snake_key']}_dict"
            lines.append(f"        val_{key} = {dict_var_name}.get('{info['original_key']}')")
            lines.append(f"        {sub_dict_name} = val_{key} if isinstance(val_{key}, dict) else {{}}")
            lines.append(f"        {info['snake_key']}_obj = {sub_cls}(")
            for sub_k, sub_info in info['sub_schema'].items():
                 lines.append(f"            {sub_info['snake_key']}={sub_dict_name}.get('{sub_info['original_key']}'),")
            lines.append("        )")
    return "\n".join(lines)

def generate_return_fields(schema, dict_var_name="doc_dict"):
    lines = []
    for key, info in schema.items():
        if info['is_nested']:
            lines.append(f"            {info['snake_key']}={info['snake_key']}_obj,")
        else:
            lines.append(f"            {info['snake_key']}={dict_var_name}.get('{info['original_key']}'),")
    return "\n".join(lines)

# --- EXECUÇÃO ---

if not sample_docs:
    print("❌ Sem dados. Verifique a conexão.")
else:
    full_schema = analyze_structure(sample_docs)
    nested_classes_code = generate_nested_classes(full_schema)

    # AQUI ESTÁ A CORREÇÃO: "from strawberry.scalars import JSON"
    final_code = f"""
import strawberry
from strawberry.scalars import JSON
from fastapi import FastAPI
from strawberry.fastapi import GraphQLRouter
from fastapi.middleware.cors import CORSMiddleware
import pymongo
import certifi
from typing import List, Optional, Any
import json
import os
import urllib.parse
from bson import json_util

raw_user = os.getenv("MONGO_USER")
raw_pass = os.getenv("MONGO_PASS")
raw_host = os.getenv("MONGO_HOST")

user = urllib.parse.quote_plus(raw_user)
passwd = urllib.parse.quote_plus(raw_pass)
MONGO_URI = f"mongodb+srv://{{user}}:{{passwd}}@{{raw_host}}/?retryWrites=true&w=majority"

try:
    client = pymongo.MongoClient(MONGO_URI, tlsCAFile=certifi.where())
    db = client["dw_atendimento"]
    coll = db["protocolos_historico"]
except:
    pass

# Função para limpar objetos do Mongo antes de enviar pro GraphQL
def sanitize_mongo(data):
    if isinstance(data, dict):
        if "$date" in data: return data["$date"]
        if "$oid" in data: return data["$oid"]
        return {{k: sanitize_mongo(v) for k, v in data.items()}}
    elif isinstance(data, list):
        return [sanitize_mongo(i) for i in data]
    return data

{nested_classes_code}

@strawberry.type
class DocumentoOficial:
    mongo_id: str
"""
    for key, info in full_schema.items():
        final_code += f"    {info['snake_key']}: {info['type']}\n"

    final_code += """
    conteudo_completo: JSON

def get_dados(limit: int = 10, cursor_date: Optional[str] = None):
    # Query Base
    query = {}

    # Lógica do Cursor (Paginação)
    if cursor_date:
        query["SP_Date_Time"] = {{"$lt": cursor_date}}

    # Lógica de Performance (Disk Use) + Ordenação
    # Importante: O .allow_disk_use(True) permite ordenar dataset gigante sem erro 292
    cursor = coll.find(query).sort("SP_Date_Time", -1).limit(limit).allow_disk_use(True)

    res = []
    for doc in cursor:
        # Pipeline de limpeza: BSON -> JSON -> Dict Limpo
        doc_str = json_util.dumps(doc)
        doc_dict = sanitize_mongo(json.loads(doc_str))

"""
    final_code += generate_resolver_logic(full_schema)
    final_code += """
        res.append(DocumentoOficial(
            mongo_id=str(doc.get('_id', '')),
"""
    final_code += generate_return_fields(full_schema)
    final_code += """
            conteudo_completo=doc_dict
        ))
    return res

@strawberry.type
class Query:
    @strawberry.field
    def explorar_dados(self, limit: int = 10, cursor_date: Optional[str] = None) -> List[DocumentoOficial]:
        return get_dados(limit, cursor_date)

schema = strawberry.Schema(query=Query)
graphql_app = GraphQLRouter(schema)
app = FastAPI()
app.include_router(graphql_app, prefix="/graphql")
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"], allow_credentials=True)
"""

    with open("api.py", "w") as f:
        f.write(final_code)

    print("\n✅ API.PY REPARADO!")
    print("1. JSON agora é o tipo oficial do Strawberry (adeus erro unhashable).")
    print("2. Paginação via 'cursor_date' ativada.")
    print("3. Proteção 'allow_disk_use' ativada.")
    print("👉 Rode o 'LIGAR REATOR' e tente o download.")

🔧 Gerando API com JSON Nativo e Cursor...

✅ API.PY REPARADO!
1. JSON agora é o tipo oficial do Strawberry (adeus erro unhashable).
2. Paginação via 'cursor_date' ativada.
3. Proteção 'allow_disk_use' ativada.
👉 Rode o 'LIGAR REATOR' e tente o download.
